### **Working With Final Data**

* Apply Previously Established Rules to new dataset
* data has one row for unique pairs of StudentID and TermCode Allowing us to classify student from past years

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('StudentData2.csv')
df.head()

,Student.ID,Term.Code,Term.UG.MU.Earned.CrHr,Term.UG.Overall.Earned.CrHr..MU.and.Non.MU.,Cum.UG.Overall.Earned.CrHr..MU.and.Non.MU.,Major.1,Major.2,Cohort.Term,Number_of_semesters
0,8071760.0,202410.0,8.0,8.0,8.0,BA Major,NaN,202410.0,1.0
1,8071760.0,202420.0,16.0,16.0,24.0,BA Major,NaN,202410.0,2.0
2,8071769.0,202410.0,15.0,38.0,38.0,CAS,NaN,202410.0,1.0
3,8071769.0,202420.0,15.0,15.0,53.0,CAS,NaN,202410.0,2.0
4,8079808.0,202410.0,15.0,15.0,15.0,CAS,NaN,202410.0,1.0


In [ ]:
#ba_df = df[(df['Major.1'] == 'BA Major') | (df['Major.2'] == 'BA Major')]
#is_df = df[(df['Major.1'] == 'IS Major')| (df['Major.2'] == 'IS Major')]

In [ ]:
# rename Credits column
df['Credits']=df['Cum.UG.Overall.Earned.CrHr..MU.and.Non.MU.'].astype(int)

In [ ]:
## drop original column
df = df.drop(columns=['Cum.UG.Overall.Earned.CrHr..MU.and.Non.MU.'])



In [ ]:
# prompt: create a new column in df called Class

# Create a new column named 'Class'
df['Class'] = ''

In [ ]:
# prompt: Senior classification: For all records Where Credits is greater than 100 or Number_of_semesters is greater than 6.5

# Senior classification: For all records Where Credits is greater than 100 or Number_of_semesters is greater than 6.5
df.loc[((df['Credits'] > 100) | (df['Number_of_semesters'] > 6.5)), 'Class'] = 'Senior'

In [ ]:
df['Class'].value_counts()

,count
Class,
,4509
Senior,2104


In [ ]:
# prompt: Junior Classification : There are two ways to classify as a junior. 1:  Number_of_semesters is greater than 1.5 but less than 3.5 AND Student has more than 74.5 credit. 2: Number_of_semesters between 3.5 and 6.5 AND credits less than 100.5

# Junior classification
df.loc[((df['Number_of_semesters'] > 1.5) & (df['Number_of_semesters'] < 3.5) & (df['Credits'] > 74.5)& (df['Credits'] < 100.5)) | ((df['Number_of_semesters'] >= 3.5) & (df['Number_of_semesters'] <= 6.5) & (df['Credits'] < 100.5)), 'Class'] = 'Junior'

In [ ]:
df['Class'].value_counts()

,count
Class,
,3064
Senior,2104
Junior,1445


In [ ]:
# prompt: Sophomore Classification: There are two ways to classify as a sophomore.   1:  Number_of_semesters is greater than 1.5 but less than 3.5 AND Student has less than 74.5 credit. 2: Number_of_semesters is less than 1.5 AND student has more than 39.5 Credits

# Sophomore classification
df.loc[((df['Number_of_semesters'] > 1.5) & (df['Number_of_semesters'] < 3.5) & (df['Credits'] < 74.5)) | ((df['Number_of_semesters'] < 1.5) & (df['Credits'] > 39.5) & (df['Credits'] < 100.5)), 'Class'] = 'Sophomore'

In [ ]:
df['Class'].value_counts()

,count
Class,
Sophomore,2273
Senior,2104
Junior,1445
,791


In [ ]:
# prompt: Freshman Classification: Number_of_semesters less than 1.5 AND credits less than 39.5

# Freshman classification
df.loc[((df['Number_of_semesters'] < 1.5) & (df['Credits'] < 39.5)), 'Class'] = 'Freshman'
df['Class'].value_counts()


,count
Class,
Sophomore,2273
Senior,2105
Junior,1445
Freshman,792


In [ ]:
# prompt: create a new column for School_Year that takes the first 4 digits of term code , if the next 2 digits are 10 then add Fall to the end of first 4, If the next two digits are 20 add Spring to the end. put a space between the numbers and word
## SchoolYear Column
# Assuming 'Term Code' column exists
df['School_Year'] = ''

# Iterate through rows and apply logic
for index, row in df.iterrows():
    term_code = str(row['Term.Code'])
    if len(term_code) >= 6:
        year = term_code[:4]
        semester_code = term_code[4:6]
        if semester_code == '10':
            df.loc[index, 'School_Year'] = year + ' Fall'
        elif semester_code == '20':
            df.loc[index, 'School_Year'] = year + ' Spring'
        else:
            df.loc[index, 'School_Year'] = year # Handle other semester codes if needed

## **Building Table for pres**

In [ ]:
# prompt:  Create a tables, rows are [Freshman, Sophomore, Junior, Senior], columns are School_Year, values are the counts of each  student in each class at the specified School_Year
## This Table has unlabeled winter and spring terms Next table is better
# Create the table
classification_counts = df.groupby(['School_Year', 'Class']).size().unstack(fill_value=0)

# Reorder the index to match the desired order
classification_counts = classification_counts.reindex(['Freshman', 'Sophomore', 'Junior', 'Senior'], axis=1)

classification_counts

Class,Freshman,Sophomore,Junior,Senior
School_Year,,,,
2015 Fall,0,2,0,0
2015 Spring,0,0,2,0
2016,0,1,0,2
2016 Fall,0,2,0,2
2016 Spring,1,1,1,2
2017,1,0,0,1
2017 Fall,2,1,2,2
2017 Spring,0,3,1,3
2018,14,23,0,2


In [ ]:
# prompt: Create a tables, rows are [Freshman, Sophomore, Junior, Senior], columns are School_Year, values are the counts of each  student in each class at the specified School_Year , Also a total column for total student each school year would be cool to have in the table, start in year 2019, Only display School_Year rows that end in Fall or Spring

import pandas as pd

# Assuming 'df' is already defined and processed as in the previous code

# Create the table
classification_counts = df.groupby(['School_Year', 'Class']).size().unstack(fill_value=0)

# Reorder the index to match the desired order
classification_counts = classification_counts.reindex(['Freshman', 'Sophomore', 'Junior', 'Senior'], axis=1)

# Add a total column
classification_counts['Total'] = classification_counts.sum(axis=1)

# Filter for years starting from 2019 and ending in Fall or Spring
classification_counts = classification_counts[classification_counts.index.str.endswith(('Fall'#, 'Spring'
))]
classification_counts = classification_counts[classification_counts.index >= '2020 Fall']

# Display the table
classification_counts

Class,Freshman,Sophomore,Junior,Senior,Total
School_Year,,,,,
2020 Fall,96,105,83,59,343
2021 Fall,66,136,82,158,442
2022 Fall,129,127,117,140,513
2023 Fall,107,181,94,185,567
2024 Fall,113,139,143,181,576


## **Steps**

* Once major is in DF we make One table for BA one for IS

In [ ]:
# split datasets by major
ba_df = df[(df['Major.1'] == 'BA Major') | (df['Major.2'] == 'BA Major')]
is_df = df[(df['Major.1'] == 'IS Major')| (df['Major.2'] == 'IS Major')]


In [ ]:

## Practice Table

# Create the table for BA majors only
classification_counts_ba = ba_df.groupby(['School_Year', 'Class']).size().unstack(fill_value=0)

# Reorder the index to match the desired order
classification_counts_ba = classification_counts_ba.reindex(['Freshman', 'Sophomore', 'Junior', 'Senior'], axis=1)

# Add a total column
classification_counts_ba['Total'] = classification_counts_ba.sum(axis=1)

# Filter for years starting from 2019 and ending in Fall or Spring
classification_counts_ba = classification_counts_ba[classification_counts_ba.index.str.endswith(('Fall','Spring'))]
classification_counts_ba = classification_counts_ba[classification_counts_ba.index >= '2021 Fall']

# Display the table
classification_counts_ba

Class,Freshman,Sophomore,Junior,Senior,Total
School_Year,,,,,
2021 Fall,7,30,39,84,160
2021 Spring,0,18,61,99,178
2022 Fall,69,39,53,77,238
2022 Spring,0,77,58,116,251
2023 Fall,70,84,45,112,311
2023 Spring,0,87,90,125,302
2024 Fall,89,89,73,108,359
2024 Spring,0,95,106,133,334


In [ ]:
## Practice Table

# Create the table for is majors only
classification_counts_is = is_df.groupby(['School_Year', 'Class']).size().unstack(fill_value=0)

# Reorder the index to match the desired order
classification_counts_is = classification_counts_is.reindex(['Freshman', 'Sophomore', 'Junior', 'Senior'], axis=1)

# Add a total column
classification_counts_is['Total'] = classification_counts_is.sum(axis=1)

# Filter for years starting from 2019 and ending in Fall or Spring
classification_counts_is = classification_counts_is[classification_counts_is.index.str.endswith(('Fall','Spring'))]
classification_counts_is = classification_counts_is[classification_counts_is.index >= '2021 Fall']

# Display the table
classification_counts_is

Class,Freshman,Sophomore,Junior,Senior,Total
School_Year,,,,,
2021 Fall,10,24,28,50,112
2021 Spring,1,14,54,64,133
2022 Fall,7,18,38,42,105
2022 Spring,0,12,41,68,121
2023 Fall,1,10,18,53,82
2023 Spring,0,3,20,64,87
2024 Fall,5,5,14,35,59
2024 Spring,0,5,9,38,52


In [ ]:
# prompt: could you flip the axis of the previous table

classification_counts_is = classification_counts_is.T
classification_counts_is

School_Year,2021 Fall,2021 Spring,2022 Fall,2022 Spring,2023 Fall,2023 Spring,2024 Fall,2024 Spring
Class,,,,,,,,
Freshman,10,1,7,0,1,0,5,0
Sophomore,24,14,18,12,10,3,5,5
Junior,28,54,38,41,18,20,14,9
Senior,50,64,42,68,53,64,35,38
Total,112,133,105,121,82,87,59,52


## **Final Tables**

In [ ]:
# prompt: Create a tables, rows are [Freshman, Sophomore, Junior, Senior], columns are School_Year, values are the counts of each  student in each class at the specified School_Year , Also a total column for total student each school year would be cool to have in the table, start in year 2019, Only display School_Year rows that end in Fall , change the display so it only show the first 4 digits of the school year

# Assuming 'df' and other variables are already defined as in the provided code

# Create the table
classification_counts = ba_df.groupby(['School_Year', 'Class']).size().unstack(fill_value=0)

# Reorder the index to match the desired order
classification_counts = classification_counts.reindex(['Freshman', 'Sophomore', 'Junior', 'Senior'], axis=1)

# Add a total column
classification_counts['Total'] = classification_counts.sum(axis=1)

# Filter for years starting from 2019 and ending in Fall
classification_counts = classification_counts[classification_counts.index.str.endswith('Fall')]
classification_counts = classification_counts[classification_counts.index >= '2021 Fall']

# Display the table, showing only the first 4 digits of the school year
classification_counts.index = classification_counts.index.str[:4]
classification_counts

Class,Freshman,Sophomore,Junior,Senior,Total
School_Year,,,,,
2021,7,30,39,84,160
2022,69,39,53,77,238
2023,70,84,45,112,311
2024,89,89,73,108,359


In [ ]:
# prompt: could you flip the axis of the previous table

classification_counts = classification_counts.T
classification_counts

School_Year,2021,2022,2023,2024
Class,,,,
Freshman,7,69,70,89
Sophomore,30,39,84,89
Junior,39,53,45,73
Senior,84,77,112,108
Total,160,238,311,359


In [ ]:

# Create the table
classification_counts = is_df.groupby(['School_Year', 'Class']).size().unstack(fill_value=0)

# Reorder the index to match the desired order
classification_counts = classification_counts.reindex(['Freshman', 'Sophomore', 'Junior', 'Senior'], axis=1)

# Add a total column
classification_counts['Total'] = classification_counts.sum(axis=1)

# Filter for years starting from 2019 and ending in Fall
classification_counts = classification_counts[classification_counts.index.str.endswith('Fall')]
classification_counts = classification_counts[classification_counts.index >= '2021 Fall']

# Display the table, showing only the first 4 digits of the school year
classification_counts.index = classification_counts.index.str[:4]
classification_counts

Class,Freshman,Sophomore,Junior,Senior,Total
School_Year,,,,,
2021,10,24,28,50,112
2022,7,18,38,42,105
2023,1,10,18,53,82
2024,5,5,14,35,59


In [ ]:
classification_counts = classification_counts.T
classification_counts

School_Year,2021,2022,2023,2024
Class,,,,
Freshman,10,7,1,5
Sophomore,24,18,10,5
Junior,28,38,18,14
Senior,50,42,53,35
Total,112,105,82,59
